In [ ]:
!pip install PyPDF2 python-dotenv streamlit openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-2

In [ ]:
pip install --upgrade streamlit-extras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.1 MB/s eta 0:00:00


In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
from dotenv import load_dotenv
import pickle
from PyPDF2 import PdfReader
from streamlit_extras.add_vertical_space import add_vertical_space
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback
import os

#sidebar contents
with st.sidebar:
  st.title("LLM Chat App")
  st.markdown('''
  ###About

  -Import Your Pdf


  -Ask Your Questions from the Pdf

  ''')


def main():
  st.header("PDF Information Extraction App")

  load_dotenv()

  #upload a pdf file
  pdf=st.file_uploader("Upload Your Pdf",type="pdf")

  if pdf is not None:
    pdf_reader=PdfReader(pdf)

    text=""
    for page in pdf_reader.pages:
      text += page.extract_text()

    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=200,
        length_function=len
        )
    chunks=text_splitter.split_text(text=text)

    #embeddings

    store_name=pdf.name[:-4]
    if os.path.exists(f"{store_name}.pkl"):
      with open(f"{store_name}.pkl","rb") as f:
        vector_store=pickle.load(f)
    else:
      embeddings=OpenAIEmbeddings()
      vector_store=FAISS.from_texts(chunks,embedding=embeddings)
      with open(f"{store_name}.pkl","wb") as f:
        pickle.dump(vector_store,f)

    #accept user questions/query
    query = st.text_input("Ask Questions About Your PDF File:")

    if query:
      docs=vector_store.similarity_search(query=query,k=3)

      llm=OpenAI(model_name="gpt-3.5-turbo")
      chain=load_qa_chain(llm=llm,chain_type="stuff")
      with get_openai_callback()as cb:
        response=chain.run(input_documents=docs,question=query)
        print(cb)
      st.write(response)






if __name__ =='__main__':
    main()

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.201.252.99:8501

npx: installed 22 in 2.244s
your url is: https://light-beans-mix.loca.lt
/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py:120: LangChainDeprecationWarning: Importing OpenAIEmbeddings from langchain.embeddings is deprecated. Please replace deprecated imports:

>> from langchain.embeddings import OpenAIEmbeddings

with new imports of:

>> from langchain_community.embeddings import OpenAIEmbeddings

  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/_api/module_import.py:120: LangChainDeprecationWarning: Importing FAISS from langchain.vectorstores is deprecated. Please replace deprecated imports:

>> from langchain.vectorstores import FAISS

with new imports of:

>> from langchain_community.vectorstores import FAISS

  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py:548: Lan

Here's the documentation for your `app.py` script:
    
### Explanation of the Script:

1. **Imports:**
   - `streamlit as st`: Used to create the Streamlit web application.
   - `load_dotenv`: Loads environment variables from a `.env` file.
   - `pickle`: Used for serializing and deserializing Python objects.
   - `PdfReader` from `PyPDF2`: Reads and extracts text from PDF files.
   - `add_vertical_space`: Adds vertical space in the Streamlit app.
   - `RecursiveCharacterTextSplitter` from `langchain.text_splitter`: Splits text into manageable chunks.
   - `OpenAIEmbeddings` from `langchain.embeddings.openai`: Generates embeddings using OpenAI.
   - `FAISS` from `langchain.vectorstores`: A library for efficient similarity search.
   - `OpenAI` from `langchain.llms`: Interacts with OpenAI's language models.
   - `load_qa_chain` from `langchain.chains.question_answering`: Loads a question-answering chain.
   - `get_openai_callback` from `langchain.callbacks`: Retrieves callback data for OpenAI interactions.
   - `os`: Provides functions for interacting with the operating system.

2. **Environment Variable Loading:**
   - `load_dotenv()`: Loads environment variables from a `.env` file into the application's environment.

3. **Sidebar Contents:**
   - The sidebar contains a title and a brief description of the application.

4. **Main Function (`main`):**
   - Displays the main header of the application.
   - Allows users to upload a PDF file.
   - Reads and extracts text from the uploaded PDF.
   - Splits the extracted text into chunks.
   - Checks if a precomputed vector store exists; if not, it generates embeddings and creates a new vector store.
   - Allows users to input a query.
   - Searches for similar documents based on the query.
   - Initializes the language model and question-answering chain.
   - Generates and displays a response to the user's query.

5. **Execution Block:**
   - Ensures the `main` function is executed when the script is run directly.

### Instructions for Use:

1. **Install Dependencies:**
   Ensure you have all the required libraries installed. You can install them using pip:
   ```bash
   pip install streamlit python-dotenv PyPDF2 langchain streamlit-extras
   ```

2. **Create a `.env` File:**
   Create a `.env` file in the same directory as `app.py` with your OpenAI API key:
   ```
   OPENAI_API_KEY=your_openai_api_key
   ```

3. **Run the Application:**
   Run the Streamlit application using the following command:
   ```bash
   streamlit run app.py
   ```

4. **Upload and Query PDF:**
   - Use the file uploader in the web app to upload a PDF file.
   - Enter your questions in the text input field to query the PDF content.

This documentation provides a comprehensive overview of the script, its functionality, and instructions for setting up and running the application.